encoder transformer层的linear2层（etl2）替换为近似矩阵乘法

In [1]:
# import sys
# sys.path.append('../../../../csi_transformer/src/')
# import backbone
# import image_segmentation
import numpy as np
import os
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
quantize_lut = False
# method = METHOD_PQ
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
nbits = 8 # 量化比特数

In [3]:
linear_name = 'etl2'
feedback_bits = 256
linear_name_full = "ex_linear2"
ncodebooks = 64 # max:512
ncentroids = 16

train_sam_num = 1000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    linearin_path_train= ''
    linearout_path_train= ''
    linearin_path_test = ''
    linearout_path_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_joined = '/data/hdr/transformer_data/joined'
    dir_train = os.path.join(dir_joined, 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join(dir_joined, 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    linearin_path_train= '%sin_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    y_train = '%s_y_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearout_path_train= '%sout_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearin_path_test = '%sin_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
    linearout_path_test = '%sout_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
else:
    raise NameError("You are running the script in a new computer %s, please define dirs" % host_name)


weightpath = '%s_w_f%i.npy' % (linear_name_full, feedback_bits)
biaspath = '%s_b_f%i.npy' % (linear_name_full, feedback_bits)
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 


In [5]:
dataset_prepare(dir_joined, linear_name_full, feedback_bits, [train_sam_num, test_sam_num], batch_size, S1 = S1_dict[linear_name])

In [6]:
est3 = mm.estFactory(X_path=linearin_path_train, W_path=weightpath, Y_path=y_train, dir= dir_train,\
                    ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits, \
                    quantize_lut = quantize_lut)


running method:  Mithral
X.shape:  (1024000, 512)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    120314.85963698717
learn_multisplits(): returning loss:  3268.6341634478745
learn_multisplits(): initial loss:    4290.847345591481
learn_multisplits(): returning loss:  1178.4152986347658
learn_multisplits(): initial loss:    72771.6794878001
learn_multisplits(): returning loss:  2417.341288024808
learn_multisplits(): initial loss:    116794.49026703188
learn_multisplits(): returning loss:  6356.5859216623285
learn_multisplits(): initial loss:    1301.899688992608
learn_multisplits(): returning loss:  353.157186591269
learn_multisplits(): initial loss:    681.9981742242628
learn_multisplits(): returning loss:  169.44290513187144
learn_multisplits(): initial loss:    2417.8987373317023
learn_multisplits(): returning loss:  638.4456759935385
learn_multisplits(): initial loss:    65178.43776484956
learn_multisplits(): returning loss:  2253.533859306866
learn_

/data/hdr/pq/bolt/experiments/python/clusterize.py:1917: UserWarning: Persisting input arguments took 2.13s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  _learn_mithral_initialization(X, ncodebooks, ncentroids=ncentroids, pq_perm_algo='start', **kwargs)


X_res mse / X mse:  0.0044924794
fitting dense lstsq to X_res
  with X_enc:(1024000, 64) Y:(1024000, 512)
fitted dense lstsq with W:(1024, 512)
X_res mse / X mse after lstsq:  0.0022449528
learn_mithral
all_centroids:
 [[[ 1.45322224e-02  3.13039660e-03 -9.69780376e-04 ... -2.10224243e-04
   -4.23338870e-03 -2.29741260e-03]
  [ 8.88461713e-03  2.15063780e-03 -5.69185824e-04 ...  1.54080026e-05
   -2.54020328e-03 -1.74793310e-03]
  [ 6.63551781e-03  1.53242936e-03 -3.26566282e-04 ... -2.76135834e-05
   -2.08812300e-03 -1.61394733e-03]
  ...
  [-1.50239293e-03 -7.96448148e-04  4.43731697e-04 ... -8.19556044e-06
    1.76027755e-03 -7.72904459e-05]
  [-1.51722052e-03 -7.90609221e-04  4.41892887e-04 ... -1.68926351e-06
    2.32751458e-03 -5.28989513e-05]
  [-1.44420355e-03 -8.30353296e-04  4.22648183e-04 ...  4.19919270e-05
    5.97597333e-03 -5.57548410e-05]]

 [[ 9.80123412e-04 -1.75773259e-03  1.28981995e-03 ...  1.05602434e-03
    6.47093914e-03  1.80635136e-03]
  [-7.18182055e-05  6.77

/data/hdr/pq/bolt/experiments/python/vquantizers.py:628: UserWarning: Persisting input arguments took 2.12s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  self.splits_lists, self.centroids = clusterize.learn_mithral(
/data/hdr/pq/bolt/experiments/python/matmul.py:328: UserWarning: Persisting input arguments took 2.37s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  est = _fitted_est_for_hparams(


In [7]:
x_test = np.load(dir_test+'/'+linearin_path_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

刚好在生成lut的代码前的质心：
 [[[ 1.45322224e-02  3.13039660e-03 -9.69780376e-04 ... -2.10224243e-04
   -4.23338870e-03 -2.29741260e-03]
  [ 8.88461713e-03  2.15063780e-03 -5.69185824e-04 ...  1.54080026e-05
   -2.54020328e-03 -1.74793310e-03]
  [ 6.63551781e-03  1.53242936e-03 -3.26566282e-04 ... -2.76135834e-05
   -2.08812300e-03 -1.61394733e-03]
  ...
  [-1.50239293e-03 -7.96448148e-04  4.43731697e-04 ... -8.19556044e-06
    1.76027755e-03 -7.72904459e-05]
  [-1.51722052e-03 -7.90609221e-04  4.41892887e-04 ... -1.68926351e-06
    2.32751458e-03 -5.28989513e-05]
  [-1.44420355e-03 -8.30353296e-04  4.22648183e-04 ...  4.19919270e-05
    5.97597333e-03 -5.57548410e-05]]

 [[ 9.80123412e-04 -1.75773259e-03  1.28981995e-03 ...  1.05602434e-03
    6.47093914e-03  1.80635136e-03]
  [-7.18182055e-05  6.77167554e-04 -1.09359808e-03 ... -9.59926459e-04
   -8.47531296e-03  6.26695168e-04]
  [ 1.74262165e-03 -1.24945131e-03  1.26184919e-03 ...  8.16344575e-04
    6.61809696e-03  3.23335920e-03]
  ...
  [-8

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL:
    np.save(os.path.join(dir_result, '%s%s_ql%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, quantize_lut, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)

[[ 0.5545293  -0.00719963  0.55830157 ...  0.29180697  0.05797084
   0.558193  ]
 [ 0.52302873  0.01604709  0.53401285 ...  0.316234    0.05467195
   0.5186102 ]
 [ 0.5737991  -0.01289414  0.5650175  ...  0.30335885  0.05428706
   0.57328147]
 ...
 [ 0.45841107  0.0850561   0.23880523 ...  0.21537417  0.21531764
   0.2863179 ]
 [ 0.36349076 -0.18696131  0.185161   ...  0.27989405  0.06499407
   0.31983444]
 [ 0.33815238  0.07204408  0.29068702 ...  0.14704172  0.06083944
   0.26588497]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
